In [3]:
import findspark
findspark.init()

In [4]:
from pyspark import Row
from pyspark.sql import SparkSession, functions, types
import sys
import json

spark = SparkSession.builder.appName('YELP').getOrCreate()
sc = spark.sparkContext

In [ ]:
# filtering data using spark dataframe
# df_business = df_business.select("business_id","name", "stars", 
#                                  "review_count", "attributes", 
#                                  "categories", "state", "city", "is_open").withColumnRenamed("stars", "stars_restaurant")

# df_business = df_business.filter((df_business['city'] == 'Toronto') & (df_business.categories.contains('Restaurants'))
#                                 & (df_business['review_count'] > 0) & (df_business['is_open'] == 1)).drop('city','is_open')

In [ ]:
business_rdd = sc.textFile('yelp_dataset/yelp_academic_dataset_business.json')\
        .map(json.loads)\
        .map(lambda x: (x['business_id'],x['name'], x['city'], int(x['stars']), int(x['review_count']),
                        int(x['is_open']), x['categories']))\
        .filter(lambda x: x[4] > 0)\
        .filter(lambda x: x[5] == 1)\
        .filter(lambda x: x[6] and len(x[6])>0 and 'Restaurants' in x[6]).cache()
business_dict = business_rdd.map(lambda x: x[0]).distinct().zipWithIndex().map(lambda x: (x[0], x[1]+1)).collectAsMap()
with open('business_dict', 'w') as f:
        f.write(json.dumps(business_dict))

business_rdd = business_rdd\
        .map(lambda x: (business_dict[x[0]], x[1], x[2], x[3], x[4], x[5], x[6]))\
        .map(lambda x: Row(business_id=x[0], name=x[1], city=x[2], stars=x[3], review_count=x[4], categories=x[6]))

business_df = spark.createDataFrame(business_rdd)
business_df.write.parquet('output/business.parquet')

In [6]:
business_dict = json.loads(open('business_dict', 'r').read())

In [7]:
#loading reviews
review_rdd = sc.textFile('yelp_dataset/yelp_academic_dataset_review.json')\
        .map(json.loads)\
        .map(lambda x: (x['review_id'],x['user_id'], x['business_id'], int(x['stars'])))\
        .filter(lambda x: x[2] in business_dict.keys()).cache()
user_list_in_reviews = review_rdd.map(lambda x: x[1]).distinct().collect()

In [ ]:
#laoding users
user_rdd = sc.textFile('yelp_dataset/yelp_academic_dataset_user.json') \
        .map(json.loads) \
        .map(lambda x: (x['user_id'], x['name'], int(x['review_count']))) \
        .filter(lambda x: x[2] > 0)\
        .filter(lambda x: x[0] in user_list_in_reviews).cache()
user_dict = user_rdd.map(lambda x: x[0]).distinct().zipWithIndex().map(lambda x: (x[0], x[1] + 1)).collectAsMap()

In [ ]:
with open('user_dict', 'w') as f:
        f.write(json.dumps(user_dict))
user_rdd = user_rdd \
        .map(lambda x: (user_dict[x[0]], x[1], x[2])) \
        .map(lambda x: Row(user_id=x[0], name=x[1], review_count=x[2]))
user_df = spark.createDataFrame(user_rdd)
user_df.write.parquet('output/user.parquet')

In [ ]:
review_dict = review_rdd.map(lambda x: x[0]).distinct().zipWithIndex().map(lambda x: (x[0], x[1]+1)).collectAsMap()
with open('review_dict', 'w') as f:
        f.write(json.dumps(review_dict))
review_rdd = review_rdd\
        .map(lambda x: (review_dict[x[0]] if x[0] in review_dict else 0,
                        user_dict[x[1]] if x[1] in user_dict else 0,
                        business_dict[x[2]] if x[2] in business_dict else 0,
                        x[3]))\
        .filter(lambda x: x[0] is not 0 and x[1] is not 0 and x[2] is not 0)\
        .map(lambda x: Row(review_id=x[0], user_id=x[1], business_id=x[2], stars=x[3]))
review_df = spark.createDataFrame(review_rdd)
review_df.write.parquet('output/review.parquet')

In [ ]:
print('This function took {} minutes to run'.format((time.time()-start)/60))